In [1]:
import os
import cv2
import mediapipe as mp
import numpy as np
import mediapipe as mp

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

import feedforward

## Setup folder and register the training data

In [2]:
DATA_PATH = os.path.join('dist') 

In [ ]:
feedforward.create_folder_if_not_exists(DATA_PATH)

In [ ]:
feedforward.clear_folder(DATA_PATH)

In [3]:
num_videos = 20
num_frames_per_video = 100
prefix = 'arrow'

In [ ]:
feedforward.register_videos(DATA_PATH, prefix, num_videos, num_frames_per_video)

## Build Model

In [ ]:
prefix_for_augmented = "augmented_"
feedforward.augment_dataset(DATA_PATH, prefix_for_augmented)

In [ ]:
feedforward.delete_files_with_prefix(DATA_PATH, prefix_for_augmented)

In [9]:
window_len = 20
TERGET_LANDMARK = mp.solutions.holistic.PoseLandmark.RIGHT_WRIST
X_training, y_training = feedforward.build_training_data(DATA_PATH, window_len, TERGET_LANDMARK)
X_training.shape, y_training.shape

Processing video #1...
Processing video #2...
Processing video #3...
Processing video #4...
Processing video #5...
Processing video #6...
Processing video #7...
Processing video #8...
Processing video #9...
Processing video #10...
Processing video #11...
Processing video #12...
Processing video #13...
Processing video #14...
Processing video #15...
Processing video #16...
Processing video #17...
Processing video #18...
Processing video #19...
Processing video #20...
Processing video #21...
Processing video #22...
Processing video #23...
Processing video #24...
Processing video #25...
Processing video #26...
Processing video #27...
Processing video #28...
Processing video #29...
Processing video #30...
Processing video #31...
Processing video #32...
Processing video #33...
Processing video #34...
Processing video #35...
Processing video #36...
Processing video #37...
Processing video #38...
Processing video #39...
Processing video #40...


((2830, 10, 2), (2830, 2))

In [10]:
model = Sequential()

model.add(LSTM(64, input_shape=(X_training.shape[1], X_training.shape[2]), activation='relu', return_sequences=True)) # LSTM layer with input shape (X_training.shape[1], X_training.shape[2]))
model.add(LSTM(64, activation='relu', return_sequences=False)) # LSTM layer with return_sequences=False for the final prediction# LSTM layer with return_sequences=False for the final prediction
model.add(Dense(2))# Dense output layer with 2 units

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

history = model.fit(X_training, y_training, batch_size=32, epochs=100, validation_split=0.2)

model.summary()

Epoch 1/100
71/71 [==============================] - 2s 8ms/step - loss: 0.0930 - accuracy: 0.7142 - val_loss: 0.0259 - val_accuracy: 0.7456
Epoch 2/100
71/71 [==============================] - 0s 5ms/step - loss: 0.0184 - accuracy: 0.8286 - val_loss: 0.0141 - val_accuracy: 0.8622
Epoch 3/100
71/71 [==============================] - 0s 5ms/step - loss: 0.0139 - accuracy: 0.8772 - val_loss: 0.0124 - val_accuracy: 0.8693
Epoch 4/100
71/71 [==============================] - 0s 6ms/step - loss: 0.0121 - accuracy: 0.8905 - val_loss: 0.0125 - val_accuracy: 0.8816
Epoch 5/100
71/71 [==============================] - 0s 5ms/step - loss: 0.0107 - accuracy: 0.9024 - val_loss: 0.0109 - val_accuracy: 0.8534
Epoch 6/100
71/71 [==============================] - 0s 5ms/step - loss: 0.0101 - accuracy: 0.9121 - val_loss: 0.0097 - val_accuracy: 0.9081
Epoch 7/100
71/71 [==============================] - 0s 5ms/step - loss: 0.0105 - accuracy: 0.9108 - val_loss: 0.0096 - val_accuracy: 0.9223
Epoch 8/100
7

## Test

In [11]:
cap = cv2.VideoCapture(feedforward.CAMERA_INDEX)
holistic = mp.solutions.holistic.Holistic()
frame_buffer = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = holistic.process(frame_rgb)

    if results.pose_landmarks:
        target_coords = results.pose_landmarks.landmark[TERGET_LANDMARK]

        height, width, _ = frame.shape

        pixel_coords_x, pixel_coords_y = int(target_coords.x * width), int(target_coords.y * height)
        cv2.circle(frame, (pixel_coords_x, pixel_coords_y), 5, (255, 0, 0), -1)

        frame_buffer.append([
            target_coords.x,
            target_coords.y,
            # target_coords.z
        ])

        if len(frame_buffer) >= window_len/2:
            prediction_input = np.array(frame_buffer)[np.newaxis, :, :]
            prediction = model.predict(prediction_input)

            prediction_x = int(prediction[0][0] * width)
            prediction_y = int(prediction[0][1] * height)
            
            cv2.arrowedLine(frame, (pixel_coords_x, pixel_coords_y), (prediction_x, prediction_y), (255, 0, 255), 2)

            frame_buffer = frame_buffer[1:]

    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 10ms/step
